In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd

from moge.network.heterogeneous_network import HeterogeneousNetwork

from sklearn.model_selection import ParameterGrid, ParameterSampler
from moge.visualization.plot_data import matrix_heatmap

In [2]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)
    
with open('moge/data/LMN_future_recall/TEST/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.test.pickle', 'rb') as file:
    network_val = pickle.load(file)
    print(len(network_val.node_list))

11579


# Set parameter space

In [3]:
from scipy.stats.distributions import uniform
parameters = {
    "d": [64, 128, 256],
    "lr": [0.001, 0.0005],
    "margin": [0.5],
    "compression_func": ["sqrt", "log", "sqrt3", "linear"],
    "negative_sampling_ratio": [1.0, 2.0, 5.0, 20.0, 40.0],
    "directed_proba": [0.0, 0.2, 0.5, 0.8, 1.0, 1.5], # undirected_weight
    "directed_distance": ["dot_sigmoid"],
    "undirected_distance": ["cosine"],
    "max_length": [8000],
    "truncating": ["random", "post"],
    
    "conv1_kernel_size": [12, 18, 26],
    "conv1_batch_norm": [True, False],
    "max1_pool_size": [3, 4, 6, 9],
    "conv2_kernel_size": [None, 2, 4, 6],
    "conv2_batch_norm": [True, False],
    "max2_pool_size": [2, 4, 6],
    "lstm_unit_size": [100, 160, 320],
    "dense1_unit_size": [None, 256, 512, 1024],
    "dense2_unit_size": [None, 256, 512],
    "source_target_dense_layers": [True, False],
    "embedding_normalization": [True, False]
}

# Train Model

In [4]:
from moge.embedding.siamese_graph_embedding import SiameseGraphEmbedding
from moge.embedding.siamese_triplet_online_embedding import SiameseOnlineTripletGraphEmbedding

# siamese = SiameseGraphEmbedding(batch_size=925, epochs=1, verbose=False)

siamese = SiameseOnlineTripletGraphEmbedding(batch_size=225, epochs=1, verbose=False)

Using TensorFlow backend.


In [ ]:
best_score = float("inf")
X_params = []
y = []

for g in ParameterSampler(parameters, n_iter=300):
    print(len(X_params), ":", g)
    siamese.set_params(**g)
    
    try:
        siamese.learn_embedding(network, network_val=network_val, multi_gpu=False, 
                            n_steps=350, validation_steps=None, histogram_freq=1,
                            tensorboard=False, rebuild_model=True,
                            seed=42)
    except Exception as e:
        print("Failed with exception:", type(e).__name__, e)
        continue
    except KeyboardInterrupt as e:
        break
    
    current_score = siamese.hist.history['val_loss'][-1]
    X_params.append(g)
    y.append(current_score)
    
    if current_score < best_score:
        best_score = current_score
        best_grid = g
        best_history = siamese.hist.history
        print("Score:", best_score, "\n")
    


0 : {'undirected_distance': 'cosine', 'truncating': 'post', 'source_target_dense_layers': False, 'negative_sampling_ratio': 2.0, 'max_length': 8000, 'max2_pool_size': 6, 'max1_pool_size': 6, 'margin': 0.5, 'lstm_unit_size': 320, 'lr': 0.001, 'embedding_normalization': True, 'directed_proba': 1.5, 'directed_distance': 'dot_sigmoid', 'dense2_unit_size': 512, 'dense1_unit_size': 512, 'd': 64, 'conv2_kernel_size': 4, 'conv2_batch_norm': False, 'conv1_kernel_size': 18, 'conv1_batch_norm': False, 'compression_func': 'linear'}
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Epoch 1/1
350/350 [==============================] - 560s 2s/step - loss: 0.2094 - val_loss: 0.8833
Score: 0.8833332657814026 

1 : {'undirected_distance': 'cosine', 'truncating': 'post', 'source_target_dense_layers': True, 'negative_sampling_ratio': 40.0, 'max_length': 8000, 'max2_pool_size': 6, 'max1_pool_size': 9, 'margin': 0.5, 'lstm_unit_size': 160, 'lr': 0.0005, 'emb

Epoch 1/1
350/350 [==============================] - 559s 2s/step - loss: 0.1753 - val_loss: 0.1343
13 : {'undirected_distance': 'cosine', 'truncating': 'random', 'source_target_dense_layers': True, 'negative_sampling_ratio': 20.0, 'max_length': 8000, 'max2_pool_size': 2, 'max1_pool_size': 9, 'margin': 0.5, 'lstm_unit_size': 320, 'lr': 0.001, 'embedding_normalization': False, 'directed_proba': 0.2, 'directed_distance': 'dot_sigmoid', 'dense2_unit_size': None, 'dense1_unit_size': 1024, 'd': 128, 'conv2_kernel_size': 4, 'conv2_batch_norm': False, 'conv1_kernel_size': 12, 'conv1_batch_norm': False, 'compression_func': 'log'}
Epoch 1/1


In [6]:
print(best_score)

0.13158462941646576


In [7]:
print(best_grid)

{'undirected_distance': 'cosine', 'truncating': 'random', 'source_target_dense_layers': True, 'negative_sampling_ratio': 20.0, 'max_length': 8000, 'max2_pool_size': 2, 'max1_pool_size': 3, 'margin': 0.5, 'lstm_unit_size': 100, 'lr': 0.0005, 'embedding_normalization': False, 'directed_proba': 0.8, 'directed_distance': 'dot_sigmoid', 'dense2_unit_size': 256, 'dense1_unit_size': 512, 'd': 256, 'conv2_kernel_size': None, 'conv2_batch_norm': False, 'conv1_kernel_size': 18, 'conv1_batch_norm': True, 'compression_func': 'log'}


In [8]:
len(X_params), len(y)

(10, 10)

In [8]:
X = pd.DataFrame(data=X_params)
X["loss"] = y
X.sort_values("loss")

,compression_func,conv1_batch_norm,conv1_kernel_size,conv2_batch_norm,conv2_kernel_size,d,dense1_unit_size,dense2_unit_size,embedding_normalization,lr,lstm_unit_size,max1_pool_size,max2_pool_size,max_length,negative_sampling_ratio,truncating,loss
8,sqrt3,True,18,True,6.0,256,512,512.0,True,0.0100,320,9,2,2200,5.0,post,0.106824
16,sqrt3,False,18,True,4.0,256,1024,NaN,True,0.0100,320,9,6,2200,5.0,post,0.113435
4,sqrt3,False,26,False,2.0,128,256,NaN,True,0.0010,320,9,2,2400,1.0,post,0.113649
5,sqrt3,False,18,True,4.0,64,512,NaN,True,0.0100,160,4,2,1800,40.0,random,0.121520
7,log,True,6,True,6.0,128,256,512.0,True,0.0010,160,9,4,2400,2.0,post,0.123277
6,sqrt,True,26,True,2.0,256,1024,NaN,True,0.0100,160,9,2,2200,2.0,random,0.125782
3,log,True,6,False,4.0,128,512,512.0,True,0.0005,100,6,2,2200,2.0,random,0.126693
33,sqrt,False,6,True,4.0,64,256,512.0,True,0.0100,160,6,6,2400,40.0,post,0.126880
24,log,False,6,False,6.0,128,1024,256.0,True,0.0100,100,4,6,2400,20.0,random,0.128660
17,sqrt3,False,26,True,6.0,128,256,512.0,True,0.0100,160,6,4,2400,5.0,post,0.133451


In [38]:
# Sampled triplet loss
X.corr()

,conv1_kernel_size,conv2_kernel_size,d,dense1_unit_size,dense2_unit_size,directed_proba,lr,lstm_unit_size,max1_pool_size,max2_pool_size,max_length,loss
conv1_kernel_size,1.000000,0.083094,-0.067887,0.101041,0.029739,0.089139,0.001046,0.171966,0.010055,-0.056790,0.013527,-0.034231
conv2_kernel_size,0.083094,1.000000,-0.016334,0.033706,0.125814,0.185991,-0.058078,0.019996,-0.013578,0.097513,0.056835,-0.154107
d,-0.067887,-0.016334,1.000000,0.028041,-0.010332,0.061115,-0.038663,-0.186344,-0.008234,-0.165212,0.010824,0.000513
dense1_unit_size,0.101041,0.033706,0.028041,1.000000,-0.038954,0.034710,-0.029428,-0.027874,0.032143,-0.114902,-0.110186,-0.015151
dense2_unit_size,0.029739,0.125814,-0.010332,-0.038954,1.000000,0.037692,-0.055607,-0.092136,-0.038954,0.062757,0.104013,-0.082427
directed_proba,0.089139,0.185991,0.061115,0.034710,0.037692,1.000000,0.039121,0.071548,-0.004573,-0.035792,0.069335,-0.084432
lr,0.001046,-0.058078,-0.038663,-0.029428,-0.055607,0.039121,1.000000,0.124277,0.032800,-0.145810,0.020267,-0.188738
lstm_unit_size,0.171966,0.019996,-0.186344,-0.027874,-0.092136,0.071548,0.124277,1.000000,0.093581,0.012975,-0.044179,-0.173416
max1_pool_size,0.010055,-0.013578,-0.008234,0.032143,-0.038954,-0.004573,0.032800,0.093581,1.000000,0.049348,0.055061,-0.003275
max2_pool_size,-0.056790,0.097513,-0.165212,-0.114902,0.062757,-0.035792,-0.145810,0.012975,0.049348,1.000000,-0.073825,-0.325119


In [10]:
# Sampled online triplet loss
X.corr()

,conv1_batch_norm,conv1_kernel_size,conv2_kernel_size,d,dense1_unit_size,dense2_unit_size,embedding_normalization,lr,lstm_unit_size,max1_pool_size,max2_pool_size,max_length,negative_sampling_ratio,loss
conv1_batch_norm,1.000000e+00,-0.081679,-6.223344e-18,-0.105209,-0.051803,-0.131466,0.013616,0.049637,-0.025464,0.015408,-0.090649,-0.027332,0.042505,-0.483913
conv1_kernel_size,-8.167858e-02,1.000000,-2.690937e-02,0.096422,-0.019064,-0.133660,-0.011525,0.081356,-0.057643,-0.001097,0.087768,0.036916,-0.053697,0.035590
conv2_kernel_size,-6.223344e-18,-0.026909,1.000000e+00,-0.111828,-0.013795,0.008061,-0.044859,0.056112,-0.034023,0.002629,-0.035868,-0.030346,0.066992,0.037505
d,-1.052085e-01,0.096422,-1.118282e-01,1.000000,0.039701,0.079836,0.107070,-0.036641,-0.097438,0.074063,-0.091483,0.064913,-0.010593,0.094543
dense1_unit_size,-5.180293e-02,-0.019064,-1.379490e-02,0.039701,1.000000,-0.048898,0.041201,-0.086884,0.025244,0.013746,-0.087694,0.056602,-0.032716,-0.012541
dense2_unit_size,-1.314657e-01,-0.133660,8.061051e-03,0.079836,-0.048898,1.000000,0.061331,-0.004586,0.053806,-0.100669,0.034264,-0.070893,0.046463,-0.061047
embedding_normalization,1.361645e-02,-0.011525,-4.485903e-02,0.107070,0.041201,0.061331,1.000000,0.088620,0.049824,-0.055081,-0.032091,-0.083150,-0.016594,0.332455
lr,4.963726e-02,0.081356,5.611161e-02,-0.036641,-0.086884,-0.004586,0.088620,1.000000,-0.041065,-0.086235,-0.083340,-0.053815,-0.018581,0.018668
lstm_unit_size,-2.546436e-02,-0.057643,-3.402283e-02,-0.097438,0.025244,0.053806,0.049824,-0.041065,1.000000,-0.053959,0.047418,-0.025911,-0.057729,0.129468
max1_pool_size,1.540841e-02,-0.001097,2.628934e-03,0.074063,0.013746,-0.100669,-0.055081,-0.086235,-0.053959,1.000000,-0.045595,0.112067,0.025152,-0.041907


In [9]:
# Sampled online triplet loss with context sampling
X.corr()

,conv1_batch_norm,conv1_kernel_size,conv2_batch_norm,conv2_kernel_size,d,dense1_unit_size,dense2_unit_size,embedding_normalization,lr,lstm_unit_size,max1_pool_size,max2_pool_size,max_length,negative_sampling_ratio,loss
conv1_batch_norm,1.000000,-0.142089,-0.079786,0.085749,0.410495,-0.073969,0.096077,NaN,-0.187283,0.193510,-0.112782,-0.202152,-0.047741,-0.003918,0.025792
conv1_kernel_size,-0.142089,1.000000,0.276534,-0.350297,-0.025695,-0.009039,-0.306970,NaN,-0.094955,-0.112095,0.268743,-0.385359,0.076977,-0.105051,0.059286
conv2_batch_norm,-0.079786,0.276534,1.000000,-0.025364,0.294364,0.068344,0.096077,NaN,0.283780,-0.076146,0.147246,0.070380,0.227504,0.073653,-0.018621
conv2_kernel_size,0.085749,-0.350297,-0.025364,1.000000,0.141377,0.076945,0.239339,NaN,0.310668,0.112913,0.044466,0.177003,0.048233,0.052893,-0.111492
d,0.410495,-0.025695,0.294364,0.141377,1.000000,0.317513,0.268484,NaN,0.246636,0.246957,0.136816,-0.089175,0.094770,-0.369831,0.151150
dense1_unit_size,-0.073969,-0.009039,0.068344,0.076945,0.317513,1.000000,-0.240214,NaN,0.090075,0.022861,-0.046204,0.000000,0.146687,-0.269631,0.201122
dense2_unit_size,0.096077,-0.306970,0.096077,0.239339,0.268484,-0.240214,1.000000,NaN,0.163934,0.124599,0.123897,-0.044412,-0.169736,-0.128688,-0.226575
embedding_normalization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lr,-0.187283,-0.094955,0.283780,0.310668,0.246636,0.090075,0.163934,NaN,1.000000,0.000254,0.173760,0.072386,-0.068492,0.184728,-0.456526
lstm_unit_size,0.193510,-0.112095,-0.076146,0.112913,0.246957,0.022861,0.124599,NaN,0.000254,1.000000,0.279556,0.069867,-0.067676,0.102058,-0.254394
